In [ ]:
import datetime as dt
import requests as req
import pandas as pd
import yfinance as yf
import sqlite3 as sqlite

In [ ]:
def GET_INDICE_COMPONENTS():
    res  = req.get('https://www.slickcharts.com/sp500', headers={ 'User-Agent':'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36', 'X-Requested-With':'XMLHttpRequest' })
    page = pd.read_html(res.content)
    tbl  = page[0]
    return tbl

SNP_500 = GET_INDICE_COMPONENTS()['Symbol']

In [ ]:
TICKERS = ['SPY','QQQ','IWM',*SNP_500.head(50)]
TODAY   = dt.date.today()

In [ ]:
def GET_YF_OPT_CHAINS(TICKERS):
    Pipe = []
    for TICKER in TICKERS:
        Asset = yf.Ticker(TICKER)
        Price = Asset.info.get('previousClose')

        for EXP in Asset.options:
            for OPT in ['calls','puts']:
                try: 
                    chain = getattr(Asset.option_chain(EXP), OPT)
                    Pipe.append(pd.DataFrame({
                        'Date':     TODAY, 
                        'Ticker':   TICKER, 
                        'Price':    Price, 
                        'Code':     chain['contractSymbol'],
                        'Opt':      OPT[0].capitalize(), 
                        'Expiry':   EXP, 
                        'Strike':   chain['strike'], 
                        'Vol':      chain['volume'], 
                        'OI':       chain['openInterest'], 
                        'Ask':      chain['ask'], 
                        'Bid':      chain['bid'], 
                        'Last':     chain['lastPrice'], 
                    }))
                except Exception as E: print(E)
            pass
        pass
    pass
    return pd.concat(Pipe, ignore_index=1)

Chains = GET_YF_OPT_CHAINS(TICKERS)
# Chains

In [ ]:
con = sqlite.connect('Filebase.db')
Chains.to_sql('Chain', con, if_exists='append', index=0)